In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

'''Train and test paths''' 
train_path = '../Final-Project-Spring24/brain_tumor_dataset/Training'
test_path = '../Final-Project-Spring24/brain_tumor_dataset/Testing'

'''Get class names''' 
classes = os.listdir(train_path)
num_classes = len(classes)
print(classes)

'''Load and preprocess data'''
def preprocess_data(data_path):
    data = []
    for i, class_name in enumerate(classes):
        class_path = os.path.join(data_path, class_name)
        for file_name in os.listdir(class_path):
            img = load_img(os.path.join(class_path, file_name), color_mode='rgb', target_size=(150, 150))
            img = img_to_array(img) / 255.0
            data.append([img, i])
    return data

train_data = preprocess_data(train_path)
test_data = preprocess_data(test_path)

'''Extract images and labels'''
train_images, train_labels = zip(*train_data)
test_images, test_labels = zip(*test_data)

'''Convert labels to categorical'''
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

'''Convert to numpy arrays and reshape images'''
train_images = np.array(train_images).reshape(-1, 150, 150, 3)
test_images = np.array(test_images).reshape(-1, 150, 150, 3)

'''Train-test split'''
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=44)

'''Data augmentation'''
data_aug = ImageDataGenerator(
    horizontal_flip=True, 
    vertical_flip=True, 
    rotation_range=20, 
    zoom_range=0.2,
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=0.1, 
    fill_mode="nearest"
)

'''Load pre-trained DenseNet201 model'''
base_model = tf.keras.applications.DenseNet201(input_shape=(150, 150, 3), include_top=False, weights='imagenet', pooling='avg')
base_model.trainable = False

'''Add custom layers for classification'''
x = tf.keras.layers.Dense(128, activation='relu')(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

'''Compile the model'''
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

'''Train the model'''
history = model.fit(data_aug.flow(X_train, y_train, batch_size=32), validation_data=(X_test, y_test), epochs=30)

'''Evaluate the model'''
y_pred = model.predict(X_test)
pred_labels = np.argmax(y_pred, axis=1)
true_labels = np.argmax(y_test, axis=1)
print(classification_report(true_labels, pred_labels))

'''Plot training history'''
plt.plot(history.history['accuracy'], 'r', label='Training Accuracy')
plt.plot(history.history['val_accuracy'], 'b', label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

'''Predict on new data'''
img_path = "../Final-Project-Spring24/brain_tumor_dataset/Testing/meningioma_tumor/image(112).jpg"
img = load_img(img_path, target_size=(150, 150))
img_array = img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)
prediction = model.predict(img_array)
predicted_class = classes[np.argmax(prediction)]
print(f"Prediction: {predicted_class}")
